# ARYA CHANDRAN - 21BDA34

1.Have you come across Grid Search Cross Validation? Fit any two models covered in previous classes and optimize them using Grid search CV.

GridSearchCV is the process of performing hyperparameter tuning in order to determine the optimal values for a given model. The performance of a model significantly depends on the value of hyperparameters. Note that there is no way to know in advance the best values for hyperparameters so ideally, we need to try all possible values to know the optimal values. Doing this manually could take a considerable amount of time and resources and thus we use GridSearchCV to automate the tuning of hyperparameters.

GridSearchCV is a function that comes in Scikit-learn’s(or SK-learn) model_selection package. This function helps to loop through predefined hyperparameters and fit your estimator (model) on your training set. So, in the end, we can select the best parameters from the listed hyperparameters.

In [90]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np #Linear Algebra
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="darkgrid")

import re
from scipy.stats import chi2_contingency

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

from subprocess import check_output
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('loan.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [4]:
train.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [5]:
train.nunique()

Loan_ID              614
Gender                 2
Married                2
Dependents             4
Education              2
Self_Employed          2
ApplicantIncome      505
CoapplicantIncome    287
LoanAmount           203
Loan_Amount_Term      10
Credit_History         2
Property_Area          3
Loan_Status            2
dtype: int64

# DATA CLEANING

In [6]:
train.Loan_Status = train.Loan_Status.map({'Y':1, 'N':0})

In [7]:
train.sample(15)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
182,LP001636,Male,Yes,0,Graduate,No,4600,0.0,73.0,180.0,1.0,Semiurban,1
25,LP001066,Male,Yes,0,Graduate,Yes,9560,0.0,191.0,360.0,1.0,Semiurban,1
45,LP001137,Female,No,0,Graduate,No,3410,0.0,88.0,NaN,1.0,Urban,1
219,LP001734,Female,Yes,2,Graduate,No,4283,2383.0,127.0,360.0,NaN,Semiurban,1
6,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,1
40,LP001119,Male,No,0,Graduate,No,3600,0.0,80.0,360.0,1.0,Urban,0
271,LP001891,Male,Yes,0,Graduate,No,11146,0.0,136.0,360.0,1.0,Urban,1
242,LP001806,Male,No,0,Graduate,No,2965,5701.0,155.0,60.0,1.0,Urban,1
91,LP001318,Male,Yes,2,Graduate,No,6250,5654.0,188.0,180.0,1.0,Semiurban,1
326,LP002068,Male,No,0,Graduate,No,4917,0.0,130.0,360.0,0.0,Rural,1


In [8]:
train.shape

(614, 13)

In [9]:
train = train.drop('Loan_ID', axis = 1)

In [10]:
train.sample(10)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
612,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,1
388,Male,Yes,0,Graduate,No,2333,2417.0,136.0,360.0,1.0,Urban,1
101,Male,No,0,Graduate,No,4843,3806.0,151.0,360.0,1.0,Semiurban,1
441,Male,Yes,0,Graduate,No,7901,1833.0,180.0,360.0,1.0,Rural,1
34,Male,No,3+,Graduate,No,12500,3000.0,320.0,360.0,1.0,Rural,0
90,Male,Yes,0,Graduate,No,2958,2900.0,131.0,360.0,1.0,Semiurban,1
163,Male,Yes,2,Graduate,No,4167,1447.0,158.0,360.0,1.0,Rural,1
320,Male,Yes,0,Graduate,No,2400,2167.0,115.0,360.0,1.0,Semiurban,1
453,Male,Yes,0,Graduate,No,2483,2466.0,90.0,180.0,0.0,Rural,1
478,Male,Yes,1,Graduate,Yes,16667,2250.0,86.0,360.0,1.0,Semiurban,1


In [11]:
train.shape

(614, 12)

In [12]:
train.isna().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [13]:
(train.isna().sum()/train.shape[0])*100

Gender               2.117264
Married              0.488599
Dependents           2.442997
Education            0.000000
Self_Employed        5.211726
ApplicantIncome      0.000000
CoapplicantIncome    0.000000
LoanAmount           3.583062
Loan_Amount_Term     2.280130
Credit_History       8.143322
Property_Area        0.000000
Loan_Status          0.000000
dtype: float64

In [14]:
train.drop_duplicates(inplace = True)

In [15]:
train.shape

(614, 12)

In [16]:
train.dropna(subset = ['Married'], inplace = True)

In [17]:
train.sample(10)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
427,Male,Yes,2,Graduate,No,5935,0.0,133.0,360.0,1.0,Semiurban,1
75,Male,No,0,Graduate,No,3750,0.0,113.0,480.0,1.0,Urban,0
226,Male,Yes,NaN,Not Graduate,Yes,4735,0.0,138.0,360.0,1.0,Urban,0
339,Female,No,0,Graduate,No,4160,0.0,71.0,360.0,1.0,Semiurban,1
266,Male,Yes,2,Graduate,No,4708,1387.0,150.0,360.0,1.0,Semiurban,1
214,Male,Yes,0,Graduate,No,3173,3021.0,137.0,360.0,1.0,Urban,1
168,Male,No,0,Graduate,No,2237,0.0,63.0,480.0,0.0,Semiurban,0
71,Male,Yes,2,Not Graduate,Yes,1875,1875.0,97.0,360.0,1.0,Semiurban,1
368,Male,Yes,1,Graduate,No,6325,0.0,175.0,360.0,1.0,Semiurban,1
45,Female,No,0,Graduate,No,3410,0.0,88.0,NaN,1.0,Urban,1


In [18]:
train['Gender'] = train['Gender'].fillna(train['Gender'].mode()[0])

In [19]:
train['Dependents'] = train['Dependents'].fillna(train['Dependents'].mode()[0])

In [20]:
train['Self_Employed'] = train['Self_Employed'].fillna(train['Self_Employed'].mode()[0])

In [21]:
train['Credit_History'] = train['Credit_History'].fillna(train['Credit_History'].mode()[0])

In [22]:
train['Property_Area'] = train['Property_Area'].fillna(train['Property_Area'].mode()[0])

In [23]:
train['LoanAmount'] = train['LoanAmount'].fillna(train['LoanAmount'].mean())

In [24]:
train['Loan_Amount_Term'] = train['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mean())

In [25]:
train.isna().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [26]:
model_df = train[['Gender','Married','Dependents','Education','Self_Employed','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','Property_Area','Loan_Status']]

In [27]:
model_df_final = pd.get_dummies(data=model_df, drop_first=True)

In [28]:
model_df_final

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Gender_Male,Married_Yes,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban
0,5849,0.0,146.369492,360.0,1.0,1,1,0,0,0,0,0,0,0,1
1,4583,1508.0,128.000000,360.0,1.0,0,1,1,1,0,0,0,0,0,0
2,3000,0.0,66.000000,360.0,1.0,1,1,1,0,0,0,0,1,0,1
3,2583,2358.0,120.000000,360.0,1.0,1,1,1,0,0,0,1,0,0,1
4,6000,0.0,141.000000,360.0,1.0,1,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,2900,0.0,71.000000,360.0,1.0,1,0,0,0,0,0,0,0,0,0
610,4106,0.0,40.000000,180.0,1.0,1,1,1,0,0,1,0,0,0,0
611,8072,240.0,253.000000,360.0,1.0,1,1,1,1,0,0,0,0,0,1
612,7583,0.0,187.000000,360.0,1.0,1,1,1,0,1,0,0,0,0,1


In [29]:
model_df_final.columns

Index(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Loan_Status', 'Gender_Male',
       'Married_Yes', 'Dependents_1', 'Dependents_2', 'Dependents_3+',
       'Education_Not Graduate', 'Self_Employed_Yes',
       'Property_Area_Semiurban', 'Property_Area_Urban'],
      dtype='object')

In [30]:
X = model_df_final[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Gender_Male', 'Married_Yes',
       'Dependents_1', 'Dependents_2', 'Dependents_3+',
       'Education_Not Graduate', 'Self_Employed_Yes',
       'Property_Area_Semiurban', 'Property_Area_Urban']]
y = model_df_final['Loan_Status']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [37]:
X_train.shape, X_test.shape

((488, 14), (123, 14))

# RANDOM FOREST CLASSIFICATION

In [38]:
clf = RandomForestClassifier(n_estimators = 100, random_state = 18).fit(X_train, y_train)

In [39]:
prediction = clf.predict(X_test)

In [43]:
confusion_matrix(y_test, prediction)

array([[17, 18],
       [ 4, 84]])

In [44]:
accuracy_score(y_test, prediction)

0.8211382113821138

In [48]:
print(classification_report(y_test,prediction))

              precision    recall  f1-score   support

           0       0.81      0.49      0.61        35
           1       0.82      0.95      0.88        88

    accuracy                           0.82       123
   macro avg       0.82      0.72      0.75       123
weighted avg       0.82      0.82      0.81       123



HYPERPARAMETER TUNING

In the case of a random forest, hyperparameters include the number of decision trees in the forest and the number of features considered by each tree when splitting a node. (The parameters of a random forest are the variables and thresholds used to split each node learned during training)

# RANDOM FOREST CLASSIFICATION USING GRID SEARCH CV

In [50]:
rfc=RandomForestClassifier(random_state=42)

In [64]:
param_grid = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt', 'log2','auto'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy'],
}

n_estimators :
This is the number of trees you want to build before taking the maximum voting or averages of predictions. Higher number of trees give you better performance but makes your code slower. You should choose as high value as your processor can handle because this makes your predictions stronger and more stable.

max_features:
These are the maximum number of features Random Forest is allowed to try in individual tree. There are multiple options available in Python to assign maximum features. Here are a few of them :

* Auto/None : This will simply take all the features which make sense in every tree.Here we simply do not put any restrictions on the individual tree.

* sqrt : This option will take square root of the total number of features in individual run. For instance, if the total number of variables are 100, we can only take 10 of them in individual tree.”log2″ is another similar type of option for max_features.

* 0.2 : This option allows the random forest to take 20% of variables in individual run. We can assign and value in a format “0.x” where we want x% of features to be considered.

max_depth: The max_depth parameter specifies the maximum depth of each tree. The default value for max_depth is None, which means that each tree will expand until every leaf is pure. A pure leaf is one where all of the data on the leaf comes from the same class.

criterion: It is a function to analyze the accuracy of the split. 

random_state :simply sets seed to the random generator, so that your train-test splits are always deterministic. If you don't set seed, it is different each time.

In [65]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8],
                         'max_features': ['sqrt', 'log2', 'auto'],
                         'n_estimators': [200, 300, 400, 500]})

In [66]:
CV_rfc.best_params_

{'criterion': 'entropy',
 'max_depth': 8,
 'max_features': 'sqrt',
 'n_estimators': 400}

In [67]:
rfc1=RandomForestClassifier(random_state=42, max_features='sqrt', n_estimators= 400, max_depth=8, criterion='entropy')

In [68]:
rfc1.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=8, max_features='sqrt',
                       n_estimators=400, random_state=42)

In [69]:
pred=rfc1.predict(X_test)

In [70]:
accuracy_score(y_test, pred)

0.8292682926829268

In [71]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.89      0.46      0.60        35
           1       0.82      0.98      0.89        88

    accuracy                           0.83       123
   macro avg       0.85      0.72      0.75       123
weighted avg       0.84      0.83      0.81       123



We can see that the model has a better accuray when implemented with GridSearch

# LOGISTIC REGRESSION

In [72]:
clf = LogisticRegression(random_state=0)

In [74]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [75]:
clf.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [76]:
y_pred = clf.predict(X_test)

In [77]:
accuracy_score(y_test, y_pred)

0.8211382113821138

In [78]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.84      0.46      0.59        35
           1       0.82      0.97      0.89        88

    accuracy                           0.82       123
   macro avg       0.83      0.71      0.74       123
weighted avg       0.82      0.82      0.80       123



# LOGISTIC REGRESSION USING GRIDSEARCH CV

In [79]:
logreg = LogisticRegression()

In [89]:
parameters = {
    'penalty' : ['l1','l2'], 
    'C'       : np.logspace(-3,3,7),
    'solver'  : ['newton-cg','lbfgs', 'liblinear','saga'],
}

penalty: imposes a penalty to the logistic model for having too many variables. This results in shrinking the coefficients of the less contributive variables toward zero. This is also known as regularization.

C : C is known as a "hyperparameter." The parameters are numbers that tell the model what to do with the characteristics, whereas the hyperparameters instruct the model on how to choose parameters.A high value of C tells the model to give more weight to the training data. A lower value of C will indicate the model to give complexity more weight at the cost of fitting the data. Thus, a high Hyper Parameter value C indicates that training data is more important and reflects the real world data, whereas low value is just the opposite of this.

solver :Scikit-learn ships with five different solvers. Each solver tries to find the parameter weights that minimize a cost function. Here are the five options:

* newton-cg — A newton method. Newton methods use an exact Hessian matrix. It's slow for large datasets, because it computes the second derivatives.

* lbfgs — Stands for Limited-memory Broyden–Fletcher–Goldfarb–Shanno. It approximates the second derivative matrix updates with gradient evaluations. It stores only the last few updates, so it saves memory. It isn't super fast with large data sets. It will be the default solver as of Scikit-learn version 0.22.0.

* liblinear — Library for Large Linear Classification. Uses a coordinate descent algorithm. Coordinate descent is based on minimizing a multivariate function by solving univariate optimization problems in a loop. In other words, it moves toward the minimum in one direction at a time. It is the default solver for Scikit-learn versions earlier than 0.22.0. It performs pretty well with high dimensionality. It does have a number of drawbacks. It can get stuck, is unable to run in parallel, and can only solve multi-class logistic regression with one-vs.-rest.

* sag — Stochastic Average Gradient descent. A variation of gradient descent and incremental aggregated gradient approaches that uses a random sample of previous gradient values. Fast for big datasets.

* saga — Extension of sag that also allows for L1 regularization. Should generally train faster than sag.

In [86]:
clf = GridSearchCV(logreg,                    # model
                   param_grid = parameters,   # hyperparameters
                   scoring='accuracy',        # metric for scoring
                   cv=10)                     # number of folds

In [91]:
clf.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2'],
                         'solver': ['lbfgs', 'liblinear', 'saga']},
             scoring='accuracy')

In [92]:
print("Tuned Hyperparameters :", clf.best_params_)

Tuned Hyperparameters : {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}


In [93]:
logreg = LogisticRegression(C = 0.01, 
                            penalty = 'l2', 
                            solver = 'liblinear')

In [94]:
logreg.fit(X_train,y_train)

LogisticRegression(C=0.01, solver='liblinear')

In [95]:
y_pred = logreg.predict(X_test)

In [97]:
accuracy_score(y_test, pred)

0.8292682926829268

In [98]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.89      0.46      0.60        35
           1       0.82      0.98      0.89        88

    accuracy                           0.83       123
   macro avg       0.85      0.72      0.75       123
weighted avg       0.84      0.83      0.81       123



We can see that the model has a better accuray when implemented with GridSearch